In [7]:
import torch 
from lavis.common.registry import registry
from lavis.models.blip2_models.blip2 import Blip2Base, disabled_train
from lavis.models.blip2_models.modeling_opt import OPTForCausalLM, OPTConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import lavis.models.blip2_models.llama as llama

device = torch.device("cuda:1")
# llama_model_path = "/home/tdt/Llama/"
llama_model_path = "/home/tdt/llama-7b/"
# llama_model_path = "/data4/tdt/3DLLM/llama-7b/"
config = llama.LLaMAConfig.from_pretrained(llama_model_path)
llama_tokenizer = llama.LLaMATokenizer.from_pretrained(llama_model_path)
llama_model = llama.LLaMAForCausalLM.from_pretrained(llama_model_path, torch_dtype=torch.float16, config=config, state_dict=None)
# llama_model.save_pretrained(save_directory="/home/tdt/llama-7b/", max_shard_size="1GB")
# llama_tokenizer.save_pretrained(save_directory="/home/tdt/llama-7b/")
# llama_model.to(device)
# llama_model.eval()




Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [1]:
import torch 
from lavis.common.registry import registry
from lavis.models.blip2_models.blip2 import Blip2Base, disabled_train
from lavis.models.blip2_models.modeling_opt import OPTForCausalLM, OPTConfig
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda:0")
chatglm_model_path = "/home/tdt/chatglm-6b/"
chatglm_tokenizer = AutoTokenizer.from_pretrained(chatglm_model_path, use_fast=False, trust_remote_code=True)
chatglm_model = AutoModel.from_pretrained(chatglm_model_path, torch_dtype=torch.float16, trust_remote_code=True).half()
chatglm_model.to(device)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0): GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
      (1): GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(i

In [2]:
prompt = "请描述一下这个三维点云。"
answer = "这个三维点云中有一张桌。"
a_ids = chatglm_tokenizer.encode(answer, add_special_tokens=True)
b_ids = chatglm_tokenizer.encode(prompt, add_special_tokens=True)
print(a_ids)
print(b_ids)

[65831, 75137, 63902, 64130, 67298, 65794, 69839, 63823, 130001, 130004]
[71492, 67194, 64258, 63907, 75137, 63902, 64130, 63823, 130001, 130004]


In [44]:
max_seq_length = 25
prompt = "请描述一下这个三维点云。"
answer = "这个三维点云中有一张桌。"
a_ids = chatglm_tokenizer.encode(answer, add_special_tokens=False)
b_ids = chatglm_tokenizer.encode(prompt, add_special_tokens=False)
input_ids = chatglm_tokenizer.build_inputs_with_special_tokens(b_ids, a_ids)
context_length = input_ids.index(chatglm_tokenizer.bos_token_id)
print("a_ids: ", a_ids)
print("b_ids: ", b_ids)
print("input_ids: ", input_ids)
print("len(input_ids): ", len(input_ids))
print(context_length)
mask_position = context_length - 1
labels = [-100] * context_length + input_ids[mask_position + 1:]
# print("labels: ", labels)
# print("len(labels): ", len(labels))

pad_len = max_seq_length - len(input_ids)
input_ids = input_ids + [chatglm_tokenizer.pad_token_id] * pad_len
labels = labels + [chatglm_tokenizer.pad_token_id] * pad_len
labels = [(-100 if l == chatglm_tokenizer.pad_token_id else l) for l in labels]
print("------------------")
print("input_ids: ", input_ids)
print("len(input_ids): ", len(input_ids))
print("labels: ", labels)
print("len(labels): ", len(labels))

a_ids:  [65831, 75137, 63902, 64130, 67298, 65794, 69839, 63823]
b_ids:  [71492, 67194, 64258, 63907, 75137, 63902, 64130, 63823]
input_ids:  [71492, 67194, 64258, 63907, 75137, 63902, 64130, 63823, 130001, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 69839, 63823, 130005]
len(input_ids):  19
9
------------------
input_ids:  [71492, 67194, 64258, 63907, 75137, 63902, 64130, 63823, 130001, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 69839, 63823, 130005, 3, 3, 3, 3, 3, 3]
len(input_ids):  25
labels:  [-100, -100, -100, -100, -100, -100, -100, -100, -100, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 69839, 63823, 130005, -100, -100, -100, -100, -100, -100]
len(labels):  25


In [37]:
prompt_token = chatglm_tokenizer.tokenize(prompt)
tgt_token = chatglm_tokenizer.tokenize(answer)
tokens = prompt_token + ["[gMASK]", "<sop>"] + tgt_token + ["<eop>"]
input_ids = chatglm_tokenizer.convert_tokens_to_ids(tokens)
context_length = input_ids.index(chatglm_tokenizer.bos_token_id)
print("input_ids: ", input_ids)
print("context_length: ", context_length)

input_ids:  [71492, 67194, 64258, 63907, 75137, 63902, 64130, 63823, 130001, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 69839, 63823, 130005]
context_length:  9


In [41]:
answer = ["这个三维点云中有一张桌子。", "这个三维点云中有一张桌子以及一把椅子。"]
tgt_token = chatglm_tokenizer(answer, add_special_tokens=False)
print("tgt_token: ", tgt_token)

tgt_token:  {'input_ids': [[65831, 75137, 63902, 64130, 67298, 65794, 74212, 63823], [65831, 75137, 63902, 64130, 67298, 65794, 74212, 63936, 66979, 77788, 63823]]}


In [43]:
from torch.nn.utils.rnn import pad_sequence
answer = ["这个三维点云中有一张桌子。", "这个三维点云中有一张桌子以及一把椅子。"]
tgt_token = chatglm_tokenizer(answer, add_special_tokens=False)
input_ids = pad_sequence([torch.tensor(chatglm_tokenizer.encode(t, add_special_tokens=False)) for t in answer], batch_first=True, padding_value=20003) 
print("input_ids: ", input_ids)
print("input_ids.shape: ", input_ids.shape)


input_ids:  tensor([[65831, 75137, 63902, 64130, 67298, 65794, 74212, 63823, 20003, 20003,
         20003],
        [65831, 75137, 63902, 64130, 67298, 65794, 74212, 63936, 66979, 77788,
         63823]])
input_ids.shape:  torch.Size([2, 11])


In [46]:
text = ["这个三维点云中有一张桌子。", "这个三维点云中有一张桌子以及一把椅子。"]
prompt = ["请描述一下这个三维点云。"] * len(text)
input_ids_list, labels_list = [], []
for t, p in zip(text, prompt):
    a_ids = chatglm_tokenizer.encode(t, add_special_tokens=False)
    b_ids = chatglm_tokenizer.encode(p, add_special_tokens=False)
    input_ids = chatglm_tokenizer.build_inputs_with_special_tokens(b_ids, a_ids)
    context_length = input_ids.index(chatglm_tokenizer.bos_token_id)
    mask_position = context_length - 1
    labels = [-100] * context_length + input_ids[mask_position + 1:]
    pad_len = max_seq_length - len(input_ids)
    input_ids = input_ids + [chatglm_tokenizer.pad_token_id] * pad_len
    labels = labels + [-100] * pad_len
    labels = [(-100 if l == chatglm_tokenizer.pad_token_id else l) for l in labels]
    input_ids_list.append(input_ids)
    labels_list.append(labels)
    print("input_ids: ", input_ids)
    print("labels: ", labels)
    print("len(input_ids): ", len(input_ids))
    print("len(labels): ", len(labels))
    print("------------------")

input_ids = torch.tensor(input_ids_list)
labels = torch.tensor(labels_list)
print("input_ids: ", input_ids)
print("labels: ", labels)
print("input_ids.shape: ", input_ids.shape)
print("labels.shape: ", labels.shape)

    

input_ids:  [71492, 67194, 64258, 63907, 75137, 63902, 64130, 63823, 130001, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 74212, 63823, 130005, 3, 3, 3, 3, 3, 3]
labels:  [-100, -100, -100, -100, -100, -100, -100, -100, -100, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 74212, 63823, 130005, -100, -100, -100, -100, -100, -100]
len(input_ids):  25
len(labels):  25
------------------
input_ids:  [71492, 67194, 64258, 63907, 75137, 63902, 64130, 63823, 130001, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 74212, 63936, 66979, 77788, 63823, 130005, 3, 3, 3]
labels:  [-100, -100, -100, -100, -100, -100, -100, -100, -100, 130004, 65831, 75137, 63902, 64130, 67298, 65794, 74212, 63936, 66979, 77788, 63823, 130005, -100, -100, -100]
len(input_ids):  25
len(labels):  25
------------------
input_ids:  tensor([[ 71492,  67194,  64258,  63907,  75137,  63902,  64130,  63823, 130001,
         130004,  65831,  75137,  63902,  64130,  67298,  65794,  74212,  63823,
         130005,     